# Analizando tiburones

In [33]:
#primero importo los módulos que necesito para trabajar
import pandas as pd
import re

In [35]:
#abro el csv que contiene el data set
sharks=pd.read_csv("input/attacks.csv", encoding='cp1252')

In [36]:
sharks_data=pd.DataFrame(shark_data)

In [47]:
sharks_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [48]:
sharks_data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [108]:
#el último field que aparece en la descripción del csv en kaggle es investigador 
#eliminamos el resto de columnas que le siguen
sharks=sharks_data[['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source']]
sharks

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
#no tiene ningún sentido analizar aquellos datos que no tienen un case number (no están registrados) por lo que los vamos a eliminar
sharks1=sharks[(sharks["Case Number"].isnull()==0)&(sharks["Case Number"]!='0')&(sharks["Case Number"]!='xx')]
sharks1

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In [132]:
#aquí analizamos si algún sexo es más proclive a ser atacado ior al de mujeres
sharks1['Sex '].value_counts()
#efectivamente nos damos cuenta que el número de hombres es mucho superior al de mujeres

M      5093
F       637
N         2
M         2
.         1
lli       1
Name: Sex , dtype: int64

In [148]:
#hay ciertas columnas que tienen muchos valores por lo que no merece la pena analizarlas, cuantos más datos más represntativo el análisis
#le añade veracidad a la muestra
sharks1.isnull().sum().sort_values(ascending=False)

Time                      3353
Species                   2837
Age                       2830
Sex                        565
Activity                   544
Location                   540
Fatal (Y/N)                539
Area                       455
Name                       210
Country                     50
Injury                      28
Investigator or Source      17
Type                         4
Year                         2
Date                         0
Case Number                  0
dtype: int64

In [133]:
#aquí podemos observar que el mayor número de los casos son unprovoked
sharks1['Type'].value_counts()

Unprovoked      4595
Provoked         573
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [142]:
sharks1[sharks1['Injury'].isnull()]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
1270,2008.04.20.a,20-Apr-2008,2008.0,Unprovoked,AUSTRALIA,New South Wales,Crescent Head,NaN,Jamie Adlington,M,NaN,NaN,UNKNOWN,NaN,"Tiger shark, 2.3m","T. Peake, GSAF"
1366,2007.07.00,Jul-2007,2007.0,Invalid,SENEGAL,NaN,NaN,Murder,Alex Takyi,NaN,NaN,NaN,NaN,NaN,Questionable Incident,"Daily Guide, 8/20/2007"
2250,1997.06.07,07-Jun-1997,1997.0,Unprovoked,BRAZIL,Rio de Janeiro,"Copacabana, Rio de Janeiro",Bathing,José Luiz Lipiani,M,NaN,NaN,UNKNOWN,NaN,NaN,"Globo, 6/9/1997"
2267,1996.12.29,29-Dec-1996,1996.0,Unprovoked,AUSTRALIA,Queensland,Coolum Beach,Surfing,Blair Hall,M,18,NaN,UNKNOWN,18h00,NaN,"The Advertiser, 12/30/1996, p.3"
2648,1990.00.00,1990,1990.0,Unprovoked,USA,Florida,"Pensacola, Escambia County",Surfing,male,M,17,NaN,UNKNOWN,NaN,NaN,NaN
3038,1981.03.00,Mar-1981,1981.0,Unprovoked,BRAZIL,Rio de Janeiro,Cabo Frio,Diving,NaN,NaN,NaN,NaN,UNKNOWN,NaN,White shark,"Globo,"
3207,1975.10.04,04-Oct-1975,1975.0,Invalid,USA,California,"Seal Beach, Orange County",Surfing,NaN,NaN,NaN,NaN,NaN,NaN,Questionable incident,Unconfirmed Report
3298,1973.08.27,27-Aug-1973,1973.0,Unprovoked,AUSTRALIA,Queensland,Palm Cove Beach,NaN,G. Cole,NaN,21,NaN,UNKNOWN,NaN,NaN,"J. Green, p.36"
3434,1969.05.22,22-May-1969,1969.0,Unprovoked,DOMINICAN REPUBLIC,NaN,NaN,Surfing,"Douglas Kuchn, Jr.",M,18,NaN,UNKNOWN,16h00,NaN,H.D.Baldridge (1994) SAF Case #1607
3457,1968.09.22,22-Sep-1968,1968.0,Invalid,USA,Florida,"Riviera Beach, Palm Beach County",Surfing,NaN,NaN,NaN,NaN,NaN,12h48,Shark involvement not confirmed,"M. Vorenberg, GSAF"


In [161]:
sharks1.isnull().sum().sort_values(ascending=False)

Time                      3353
Species                   2837
Age                       2830
Sex                        565
Activity                   544
Location                   540
Fatal (Y/N)                539
Area                       455
Name                       210
Country                     50
Injury                      28
Investigator or Source      17
Type                         4
Year                         2
Date                         0
Case Number                  0
dtype: int64

In [170]:
#rename sharks and sort by descending
sharks_country=sharks1.groupby(['Country']).agg({'Country':'count'})
sharks[]

ValueError: 'Country' is both an index level and a column label, which is ambiguous.